In [1]:
import numpy as np
#np.set_printoptions(threshold = np.nan, linewidth = 115)
import pickle

# Load with pickle instead of processing images again
training_img_1 = pickle.load(open('1vAll_img_res_Infiltration_1st_half.p', 'rb'))
training_img_2 = pickle.load(open('1vAll_img_res_Infiltration_2nd_half.p', 'rb'))

In [2]:
training_img_one = np.append(training_img_1, training_img_2, axis=0)

In [3]:
training_img_3 = pickle.load(open('1vAll_img_res_Infiltration_3rd_half.p', 'rb'))
training_img_4 = pickle.load(open('1vAll_img_res_Infiltration_4th_half.p', 'rb'))

In [4]:
training_img_two = np.append(training_img_3, training_img_4, axis=0)

In [5]:
training_img = np.append(training_img_one, training_img_two, axis=0)

training_img.shape

(36298, 256, 256, 1)

In [6]:
labels_1 = pickle.load(open('1vAll_labels_res_Infiltration_1st_half.p', 'rb'))
labels_2 = pickle.load(open('1vAll_labels_res_Infiltration_2nd_half.p', 'rb'))
labels_3 = pickle.load(open('1vAll_labels_res_Infiltration_3rd_half.p', 'rb'))
labels_4 = pickle.load(open('1vAll_labels_res_Infiltration_4th_half.p', 'rb'))

In [7]:
training_labels = np.append(labels_1, np.append(labels_2, np.append(labels_3, labels_4, axis = 0), axis = 0), axis = 0)

In [8]:
test_img = pickle.load(open('1vAll_test_img.p', 'rb'))
test_labels = pickle.load(open('1vAll_test_labels.p', 'rb'))

print('Labels shape: ', training_labels.shape)
print('Length of test_labels: ', len(test_labels))
print('No. of Infiltration Diagnoses: ', sum(training_labels))

Labels shape:  (36298,)
Length of test_labels:  3490
No. of Infiltration Diagnoses:  18149


In [9]:
import keras
from keras import models, optimizers, layers, regularizers, metrics, losses
from keras.layers.advanced_activations import LeakyReLU, PReLU, ELU, ReLU, ThresholdedReLU
from keras.layers.core import Dense, Dropout, SpatialDropout2D, Activation
from keras.layers.convolutional import Conv2D, SeparableConv2D
from keras.models import model_from_json, Sequential

import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config = config)

import matplotlib.pyplot as plt

IMG_SIZE = 256

# Save Comparison model
def save_model(model_name, hist_str, model_str):

    pickle.dump(model_name.history, open('Training Histories/'+ hist_str + '.p', 'wb'))
    
    print("Saved " + hist_str + " to Training Histories folder")
    
    # serialize model to JSON
    model_name = model.to_json()
    with open("CNN Models/" + model_str + ".json", "w") as json_file:
        json_file.write(model_name)

    # serialize weights to HDF5
    model.save_weights("CNN Models/" + model_str + ".h5")
    print("Saved " + model_str + " and weights to CNN Models folder")
    
# Load model architecture and weights NOTE: must compile again
def load_model():
    model_str = str(input("Name of model to load: "))

    # load json and create model
    json_file = open('CNN Models/' + model_str + '.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights("CNN Models/" + model_str + ".h5")
    print("Loaded " + model_str + " and weights from CNN Models folder")
    
    return loaded_model
    
# Load history object
def load_history():
    hist_str = str(input("Name of history to load: "))

    loaded_history = pickle.load(open('Training Histories/' + hist_str + '.p', 'rb'))
    
    print("Loaded " + hist_str + " from Training Histories folder")
    
    return loaded_history

Infiltration_class_weight = [{0: 1, 1: 12.938}]

### Custom Loss function
#from keras import backend as K

#def LSEP(true_label, pred_label):
#    return K.log(1 + K.sum(K.exp(true_label - pred_label)))


from sklearn.metrics import confusion_matrix
import itertools

def plot_confusion_matrix(cm, classes,
                         normalize = False,
                         title = 'Confusion Matrix',
                         cmap=plt.cm.Blues):
    _fontsize = 'xx-large'
    
    if normalize:
        cm = cm.astype('float') / cm.sum(axis = 1)[:, np.newaxis]
        print("Normalized Confusion Matrix")
    else:
        print("Confusion Matrix without Normalization")
    
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title, fontsize=_fontsize)
    cb = plt.colorbar()
    cb.ax.tick_params(labelsize=15)
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation = 45, fontsize=_fontsize) 
    plt.yticks(tick_marks, classes, fontsize=_fontsize)
    
    fmt = '.2f' if normalize else 'd'
    thresh = cm.min() + 0.2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                horizontalalignment = 'center',
                color='white' if cm[i, j] > thresh else 'black', 
                fontsize=_fontsize)
    
    plt.tight_layout()
    plt.ylabel('True Labels', fontsize=_fontsize)
    plt.xlabel('Predicted Labels', fontsize=_fontsize)

# Metric Analysis
def _1vAll_accuracy(y_test, pred):
    
    pred = np.squeeze(pred, axis = -1)
    pred = np.round_(pred)
    pred = pred.astype(dtype = 'uint8')
    
    ft = pred == y_test
    
    accuracy = sum(ft)/len(ft)
        
    print('\t Complete Label Accuracy: %.2f' % round((accuracy * 100), 2), '%')
    
    print('Sum of Fully Correct Predictions: ', sum(ft))
    print('\t\t    Total Labels: ', len(ft))
    
    return accuracy

Using TensorFlow backend.


### One Vs. All

In [10]:
# Last model: RNN_Test
model = load_model()
model.summary()

model.compile(optimizer = optimizers.RMSprop(lr = 1e-3), 
              loss = 'binary_crossentropy', 
              metrics = ['accuracy'])

model_obj = model.fit(training_img, training_labels, 
                      epochs = 100, initial_epoch = 25, 
                      validation_split = (1 / 8), 
                      batch_size = 64, verbose = 2 )

Predictions = model.predict(test_img)

Accuracy = _1vAll_accuracy(test_labels, Predictions)

history_str = 'ResNet_100e_history'
model_str   = 'ResNet_100e'
    
save_model(model_obj, history_str, model_str)

acc = model_obj.history['acc']
val_acc = model_obj.history['val_acc']
loss = model_obj.history['loss']
val_loss = model_obj.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.figure(figsize=(15, 10))
plt.plot(epochs, acc, 'bd', label='Training Accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation Accuracy')
plt.title('Training and Validation Accuracy', fontsize='xx-large')
plt.xticks(fontsize='xx-large')
plt.xlabel('Epochs', fontsize='xx-large')
plt.yticks(fontsize='xx-large')
plt.ylabel('Accuracy', fontsize='xx-large')
plt.legend(fontsize='xx-large')
plt.show()
plt.figure(figsize=(15, 10))
plt.plot(epochs, loss, 'rd', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and Validation Loss', fontsize='xx-large')
plt.xticks(fontsize='xx-large')
plt.xlabel('Epochs', fontsize='xx-large')
plt.yticks(fontsize='xx-large')
plt.ylabel('Loss', fontsize='xx-large')
plt.legend(fontsize='xx-large')
plt.show()

# Plot Confusion Matrix
cm = confusion_matrix(test_labels, np.round_(Predictions))
cm_plot_labels = ['Not Infiltration', 'Infiltration']

plt.clf()
fig = plt.figure(figsize = (20, 10))
plot_confusion_matrix(cm, cm_plot_labels, title = model_str, normalize=True)
#save_plt(fig)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 1)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 256, 256, 32) 320         input_1[0][0]                    
__________________________________________________________________________________________________
re_lu_1 (ReLU)                  (None, 256, 256, 32) 0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 128, 128, 32) 0           re_lu_1[0][0]                    
__________________________________________________________________________________________________
conv2d_2 (

Train on 31760 samples, validate on 4538 samples
Epoch 1/200
 - 175s - loss: 6.3361 - acc: 0.0924 - val_loss: 16.1152 - val_acc: 0.0000e+00
Epoch 2/200
 - 164s - loss: 8.2164 - acc: 0.0060 - val_loss: 1.1921e-07 - val_acc: 0.0000e+00
Epoch 3/200
 - 165s - loss: 9.1101 - acc: 0.0000e+00 - val_loss: 1.1921e-07 - val_acc: 0.0000e+00
Epoch 4/200
 - 165s - loss: 9.1101 - acc: 0.0000e+00 - val_loss: 1.1921e-07 - val_acc: 0.0000e+00
Epoch 5/200
 - 165s - loss: 9.1101 - acc: 0.0000e+00 - val_loss: 1.1921e-07 - val_acc: 0.0000e+00
Epoch 6/200
 - 164s - loss: 9.1101 - acc: 0.0000e+00 - val_loss: 1.1921e-07 - val_acc: 0.0000e+00
Epoch 7/200
 - 165s - loss: 9.1101 - acc: 0.0000e+00 - val_loss: 1.1921e-07 - val_acc: 0.0000e+00
Epoch 8/200
 - 164s - loss: 9.1101 - acc: 0.0000e+00 - val_loss: 1.1921e-07 - val_acc: 0.0000e+00
Epoch 9/200
 - 164s - loss: 9.1101 - acc: 0.0000e+00 - val_loss: 1.1921e-07 - val_acc: 0.0000e+00
Epoch 10/200
 - 166s - loss: 9.1101 - acc: 0.0000e+00 - val_loss: 1.1921e-07 - v

Epoch 84/200
 - 164s - loss: 9.1101 - acc: 0.0000e+00 - val_loss: 1.1921e-07 - val_acc: 0.0000e+00
Epoch 85/200
 - 164s - loss: 9.1101 - acc: 0.0000e+00 - val_loss: 1.1921e-07 - val_acc: 0.0000e+00
Epoch 86/200
 - 164s - loss: 9.1101 - acc: 0.0000e+00 - val_loss: 1.1921e-07 - val_acc: 0.0000e+00
Epoch 87/200
 - 163s - loss: 9.1101 - acc: 0.0000e+00 - val_loss: 1.1921e-07 - val_acc: 0.0000e+00
Epoch 88/200
 - 164s - loss: 9.1101 - acc: 0.0000e+00 - val_loss: 1.1921e-07 - val_acc: 0.0000e+00
Epoch 89/200
 - 164s - loss: 9.1101 - acc: 0.0000e+00 - val_loss: 1.1921e-07 - val_acc: 0.0000e+00
Epoch 90/200
 - 164s - loss: 9.1101 - acc: 0.0000e+00 - val_loss: 1.1921e-07 - val_acc: 0.0000e+00
Epoch 91/200
 - 164s - loss: 9.1101 - acc: 0.0000e+00 - val_loss: 1.1921e-07 - val_acc: 0.0000e+00
Epoch 92/200
 - 164s - loss: 9.1101 - acc: 0.0000e+00 - val_loss: 1.1921e-07 - val_acc: 0.0000e+00
Epoch 93/200
 - 164s - loss: 9.1101 - acc: 0.0000e+00 - val_loss: 1.1921e-07 - val_acc: 0.0000e+00
Epoch 94/2

 - 164s - loss: 9.1101 - acc: 0.0000e+00 - val_loss: 1.1921e-07 - val_acc: 0.0000e+00
Epoch 167/200
 - 164s - loss: 9.1101 - acc: 0.0000e+00 - val_loss: 1.1921e-07 - val_acc: 0.0000e+00
Epoch 168/200
 - 163s - loss: 9.1101 - acc: 0.0000e+00 - val_loss: 1.1921e-07 - val_acc: 0.0000e+00
Epoch 169/200
 - 164s - loss: 9.1101 - acc: 0.0000e+00 - val_loss: 1.1921e-07 - val_acc: 0.0000e+00
Epoch 170/200
 - 165s - loss: 9.1101 - acc: 0.0000e+00 - val_loss: 1.1921e-07 - val_acc: 0.0000e+00
Epoch 171/200
 - 164s - loss: 9.1101 - acc: 0.0000e+00 - val_loss: 1.1921e-07 - val_acc: 0.0000e+00
Epoch 172/200
 - 165s - loss: 9.1101 - acc: 0.0000e+00 - val_loss: 1.1921e-07 - val_acc: 0.0000e+00
Epoch 173/200


KeyboardInterrupt: 